In [1]:
import spacy

nlp = spacy.load('en')
import spacy
from spacy import displacy
from collections import Counter
import re
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn import *
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
import string
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings

warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.remove('no')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', '-PRON-'])

nlp = spacy.load('en', disable = ['parser'])
punctuations = string.punctuation

print('importing news')
#df = pd.read_csv(r'/Users/msyeom/Google Drive/Machine Learning for Global Risk/Tr Corpus/Brexit.csv',header=0, encoding='utf-8').dropna(subset = ['body'])
print('importing documents')

# We drop a row only if both body and headline are NaN.
df0 = pd.read_csv(r'./Raw data/BREXIT_RTRS_2015.csv', header=0,
                  encoding='utf-8').dropna(subset=['body', 'headline'], how='all')
df1 = pd.read_csv(r'./Raw data/BREXIT_RTRS_2016.csv', header=0,
                  encoding='utf-8').dropna(subset=['body', 'headline'], how='all')
df2 = pd.read_csv(r'./Raw data/BREXIT_RTRS_2017.csv', header=0,
                  encoding='utf-8').dropna(subset=['body', 'headline'], how='all')
df3 = pd.read_csv(r'./Raw data/BREXIT_RTRS_2018.csv', header=0,
                  encoding='utf-8').dropna(subset=['body', 'headline'], how='all')
df4 = pd.read_csv(r'./Raw data/BREXIT_RTRS_2019.csv', header=0,
                  encoding='utf-8').dropna(subset=['body', 'headline'], how='all')

df = df0.append([df1, df2, df3, df4], ignore_index=True, sort=True)

# Fill na with "" to prevent errors during string-manipulation (cleaning)
df['body'].fillna("", inplace=True)
df['headline'].fillna("", inplace=True)

# We drop if headline AND body are duplicated.
df = df.drop_duplicates(['id']) 
df = df.drop_duplicates((['headline', 'body']))


df.head()

#df.columns = ['Publication Number','Assignee - Standardized', 'Assignee - DWPI','Title','Abstract','Claims','Application Year','cecj']
df.versionCreated = pd.to_datetime(df.versionCreated)

df = df[~df.headline.str.startswith('MEDIA-')] # remove MEDIA articles

def body_filt(txt):
    txt = re.sub("[\(\[].*?[\)\]]", "", txt).replace("\'", " ").replace('\r', '').replace('\n', '')
    txt = re.sub(r"http\S+", "",txt).strip()
    txt = re.sub('<[^<]+?>', '', txt)
    return txt

df['body'] = df.body.apply(body_filt) # remove [] <> () + links 

#video link articles's description
df.loc[df.body.str.startswith('Click the following link to watch video'), 'body'] = df[df.body.str.startswith('Click the following link to watch video')].body\
    .apply(lambda x : x.split('Description:')[1])\
    .apply(lambda x : x.split('Short Link:')[0])\

#remove other articles
df.loc[df.headline.str.startswith('TAKE A LOOK-'), 'body'] = df[df.headline.str.startswith('TAKE A LOOK-')].body.apply(lambda x : x.split('Click on')[0])
df['body'] = df['body'].apply(lambda x : " ".join(x.split()))
df

keyword = ['article 50', 'backstop', 'canada model', 'canada-style', 'customs union', 'hard border', 'hard brexit', 'soft brexit','meaningful vote', 'no deal', 'no-deal', 
           'withdrawal agreement', 'wto', 'world trade organization', 'world trade organisation','single market', 'fta', 'free-trade agreement', 'divorce bill', 'free movement', 'norway', 'norwegian', 'norway style',
           "eea", "court of justice of the european union", 'european court of justice', 'CJEU']

print('tokenizing and filtering')
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', '-PRON-'])
stop_words.remove('no')
def tokening(doc):
    doc = re.sub(' +', ' ',doc)
    doc = re.sub('[^A-Za-z0-9]+', ' ', doc)
    doc = re.sub(r'\d+', '', doc)
    doc = word_tokenize(doc)
    return [n for n in doc if not n in stop_words]

tok = df.body.apply(tokening)
join_tok = tok.apply(lambda x : " ".join(x))
del tok
nlp_join_tok = join_tok.apply(nlp)
del join_tok
useless = ['DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL', 'GPE']

def ent_filt(doc):
    return [i.lemma_ for i in doc.ents if not i.label_ in useless]
ent_list = nlp_join_tok.apply(ent_filt)

def ent_filt(doc):
    return [i.lemma_ for i in doc if i.lemma_ not in stop_words]
print('entity processing')
non_ent_list = nlp_join_tok.apply(ent_filt)
del nlp_join_tok
df_tf = pd.DataFrame()
df_tf['named entity'] = ent_list
df_tf['non-named entity'] = non_ent_list
df_tf['pre ne']=df.body.apply(lambda x : [key for key in keyword if key in x.lower()])
print('tf increasing')

n=1
m=5
def chek(df_tf):
    return np.append(np.tile(df_tf['named entity'],n), df_tf['non-named entity'])
def chek2(df_tf):
    if df_tf['pre ne'] != np.nan:
        return np.append(np.tile(df_tf['pre ne'],m), df_tf['union'])
    else:
        returndf_tf['union']
df_tf['union'] = df_tf.apply(chek, axis =1)
df_tf['union2'] = df_tf.apply(chek2, axis =1)



id2word = corpora.dictionary.Dictionary(df_tf['union2'])
corpus = df_tf['union2'].apply(lambda x : id2word.doc2bow(x))

print('mallet importing')

import os 
os.environ.update({'MALLET_HOME':r"/Users/msyeom/Downloads/mallet-2.0.8"})
mallet_path = '/Users/msyeom/Downloads/mallet-2.0.8/bin/mallet' # update this path
MALLET_HOME=r'/Users/msyeom/Downloads/mallet-2.0.8'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=8, id2word=id2word)
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

print('computing score')

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=df_tf['union2'], dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


import gensim    
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, id2word)
vis

ModuleNotFoundError: No module named 'spacy'

# 1. Save the corpus and dictionary for future use.

Source: https://datascienceplus.com/topic-modeling-in-python-with-nltk-and-gensim/

In [12]:
# We convert to bag-of-words corpus
#import pickle
#pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [18]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [id2word.doc2bow(doc) for doc in df_tf['union2']]
corpora.MmCorpus.serialize('corpus.mm', doc_term_matrix)

In [13]:
# save the dictionary and corpus for future use.
id2word.save('id2word.gensim')

In [15]:
# save the model object that LDA algorithm, i.e., malletmodel2ldamodel() created.
model.save('model.gensim')

In [10]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

SyntaxError: invalid syntax (<ipython-input-10-fd38a53b19e8>, line 2)

In [5]:
pyLDAvis.save_html(vis, "vis.html")

In [11]:
def format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=df.body.tolist()):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=df.body.tolist())

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)



,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,6.0,0.2670,"eu, deal, union, brexit, agreement, british, b...","LONDON, Jan 2 - Britain wants to include finan..."
1,1,2.0,0.1640,"brexit, percent, year, economy, rate, month, b...","Jan 2 - Companies seeking to ""bulk up"" to offs..."
2,2,7.0,0.5165,"usd, gbp, high, low, risk, week, fed, eur, dol...","* Cable vols off pre X-mas and long term lows,..."
3,3,7.0,0.3312,"usd, gbp, high, low, risk, week, fed, eur, dol...",* 0.8919 = five-week high for EUR/GBP after st...
4,4,5.0,0.3539,"british, company, uk, group, business, britain...",* U.S. FDA approves 46 novel drugs versus 22 i...
5,5,7.0,0.5806,"usd, gbp, high, low, risk, week, fed, eur, dol...",Japan holiday today/Wed and next Monday dampen...
6,6,5.0,0.2983,"british, company, uk, group, business, britain...","DUBLIN, Jan 2 - Ryanair recently applied for a..."
7,7,2.0,0.3152,"brexit, percent, year, economy, rate, month, b...","LONDON, Jan 2 - Yields on benchmark British gi..."
8,8,5.0,0.2799,"british, company, uk, group, business, britain...",Jan 3 - Britain s FTSE 100 index is seen openi...
9,9,5.0,0.3341,"british, company, uk, group, business, britain...",Jan 3 - Britain s FTSE 100 index is seen openi...


In [12]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

In [13]:
# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.7898,"eu, european, brexit, london, britain, financi...",* French watchdog says EU must develop own cap...
1,1.0,0.8000,"brexit, deal, party, vote, minister, eu, there...","LONDON, Nov 16 - Some lawmakers in Prime Minis..."
2,2.0,0.7918,"brexit, percent, year, economy, rate, month, b...","* Factory output falls 0.2 pct in Feb, first d..."
3,3.0,0.8579,"trump, british, britain, trade, russian, presi...",* UK names suspects as Alexander Petrov and Ru...
4,4.0,0.9415,"european, percent, market, stock, share, europ...",* Euro zone shares hit ten year high * STOXX 6...


In [14]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,6.0,"eu, deal, union, brexit, agreement, british, b...",1296.0,0.1004
1,2.0,"brexit, percent, year, economy, rate, month, b...",2517.0,0.1950
2,7.0,"usd, gbp, high, low, risk, week, fed, eur, dol...",1244.0,0.0964
3,7.0,"usd, gbp, high, low, risk, week, fed, eur, dol...",958.0,0.0742
4,5.0,"british, company, uk, group, business, britain...",571.0,0.0442
5,7.0,"usd, gbp, high, low, risk, week, fed, eur, dol...",1573.0,0.1219
6,5.0,"british, company, uk, group, business, britain...",2460.0,0.1906
7,2.0,"brexit, percent, year, economy, rate, month, b...",2290.0,0.1774
8,5.0,"british, company, uk, group, business, britain...",NaN,NaN
9,5.0,"british, company, uk, group, business, britain...",NaN,NaN
